## ToBigs 5주차

### Vision Advanced 과제

#### 문제 1.

Object Detection 에는 2-stage model 과 1-stage model이 존재합니다.  
각 유형에 해당하는 model을 하나씩 선정하여 설명하세요. (단, 세션 시간에 설명한 모델과 아래 제시된 모델은 제외할 것)

In [ ]:
#### 답안 작성

# 1-stage model: RetinaNet

## 클래스 불균형 문제를 해결하기 위해 Focal Loss를 도입한 것을 특징으로 한다. 다른 모델과 기본적인 골자는 똑같지만 분류 문제를 더욱 잘 해결하기 위해 다른 모델과 다른 loss function을 활용하는게 특징이다.
## 클래스 불균형 문제는 사진에는 객체보다 배경이 더 많기 때문에 배경의 class를 예측하기는 상대적으로 쉽지만 객체를 예측하는 것은 어렵기에 학습이 배경에 집중되는 문제이다.
## Focal loss는 배경과 같은 쉬운 예측은 가중치를 낮추고 객체와 같이 어려운 예측의 가중치를 높임으로써 객체 탐지에 대한 성능을 향상할 수 있다.

# 2-stage model: Mask R-CNN

## Mask R-CNN는 객체의 경계를 픽셀 단위까지 구분(Instance Segmentation)하기 위해 Mask Branch라는 과정을 거친다.
## 기존 방식대로 객체가 있을 것이라 추정되는 Region Proposal을 특정 크기의 feature map으로 변환 후 해당 지역의 객체가 무엇일지 분류한다.
## 이후 해당 영역에서 픽셀 단위 영역마다 분류된 객체가 속하는지 탐지하여 결과적으로 객체의 경계를 세밀하게 탐지한다.
## 이때 픽셀마다 Binary Mask(객체가 속하면 1, 아니면 0) 형식으로 값을 반환하며 1이면 해당 픽셀이 예측한 객체가 속할 것으로 판단하고 0이면 배경으로 판단한다.

#### 문제 2.

아래 제시된 FasterRCNN 과 YOLOv5 를 각각 실행합니다.  
실행 결과를 제시하고 두 모델 사이의 차이점을 실행 결과에 근거하여 설명하세요.

In [1]:
## Package Import

import torch
from torch.utils.data import DataLoader
from torchvision import models, transforms
from torchvision.datasets import CocoDetection
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import numpy as np
import time
import warnings

warnings.filterwarnings("ignore")

In [ ]:
## Data Download

import os
os.makedirs('./data/', exist_ok=True)
os.makedirs('./data/images/', exist_ok=True)

!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip
!unzip annotations_trainval2017.zip -d ./data/
!wget http://images.cocodataset.org/zips/val2017.zip
!unzip val2017.zip -d ./data/images/

In [3]:
## FasterRCNN

# 경로 설정
image_dir = "./data/images/val2017/"
json_path = "./data/annotations/instances_val2017.json"

# Transform 설정
transform = transforms.Compose([
    transforms.ToTensor()
])

# Dataset과 DataLoader 설정
dataset = CocoDetection(root=image_dir, annFile=json_path, transform=transform)
data_loader = DataLoader(dataset, batch_size=4, shuffle=False, num_workers=2, collate_fn=lambda x: tuple(zip(*x)))

# 모델 로드 및 설정
model = models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

# 모델의 클래스 수를 3개로 설정 (배경 포함해서 0, 2, background)
num_classes = 3  # COCO의 경우 background 포함 (0, 2 두 개의 클래스 + 배경)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

# GPU 사용 가능 시 GPU로 설정
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# 모델을 평가 모드로 설정
model.eval()

# 정확도 계산을 위한 변수
total_correct = 0
total_predictions = 0

# 소요 시간 측정 시작
start_time = time.time()

# 100장 예측 속도를 비교
cnt = 0

with torch.no_grad():
    for images, targets in data_loader:
        cnt += 1
        images = list(image.to(device) for image in images)

        # 각 이미지의 타깃을 적절히 변환하여 GPU로 전송
        processed_targets = []
        for target in targets:
            processed_target = {}
            processed_target['boxes'] = torch.tensor([ann['bbox'] for ann in target]).to(device)
            processed_target['labels'] = torch.tensor([ann['category_id'] for ann in target]).to(device)
            processed_targets.append(processed_target)

        # 모델 예측
        outputs = model(images)

        for i, output in enumerate(outputs):
            pred_labels = output['labels'].cpu().numpy()
            true_labels = processed_targets[i]['labels'].cpu().numpy()

            # 예측 수가 실제 라벨 수보다 많은 경우 예측 수를 잘라냄
            if len(pred_labels) > len(true_labels):
                pred_labels = pred_labels[:len(true_labels)]

            # 예측 레이블과 실제 레이블이 얼마나 일치하는지 확인
            correct = np.sum(pred_labels == true_labels[:len(pred_labels)])
            total_correct += correct
            total_predictions += len(true_labels)

        if cnt == 100:
            break

# 소요 시간 측정 종료
end_time = time.time()

# 소요 시간 및 정확도 출력
time_taken = end_time - start_time
accuracy = total_correct / total_predictions if total_predictions > 0 else 0

print(f"소요 시간: {time_taken:.2f}초")
print(f"정확도: {accuracy * 100:.2f}%")

loading annotations into memory...
Done (t=1.14s)
creating index...
index created!


Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100%|██████████| 160M/160M [00:01<00:00, 86.8MB/s]


소요 시간: 58.27초
정확도: 20.46%


In [4]:
## YOLOv5

# 경로 설정
image_dir = "./data/images/val2017/"
json_path = "./data/annotations/instances_val2017.json"

# Transform 설정
transform = transforms.Compose([
    transforms.Resize((640, 640)),  # YOLOv5의 기본 입력 크기
    transforms.ToTensor()
])

# Dataset과 DataLoader 설정
dataset = CocoDetection(root=image_dir, annFile=json_path, transform=transform)
data_loader = DataLoader(dataset, batch_size=1, shuffle=False, num_workers=2)

# 모델 로드 (PyTorch Hub에서 COCO로 사전 학습된 YOLOv5 모델 로드)
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# GPU 사용 가능 시 GPU로 설정
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
model.eval()

# 정확도 계산을 위한 변수
total_correct = 0
total_predictions = 0

# 소요 시간 측정 시작
start_time = time.time()

# 100장 예측 속도를 비교
cnt = 0

with torch.no_grad():
    for images, targets in data_loader:
        cnt += 1
        images = images.to(device)

        # 모델 예측
        outputs = model(images)

        # 예측된 바운딩 박스와 클래스 정보 추출
        for output in outputs:
            pred_labels = output[:, 5:].argmax(1).cpu().numpy()  # 예측된 클래스 레이블

            # 실제 라벨을 추출하고, 예측된 라벨과 비교
            true_labels = [t['category_id'].item() for t in targets]

            # 예측된 라벨과 실제 라벨이 일치하는지 확인
            correct = np.sum(pred_labels[:len(true_labels)] == true_labels)
            total_correct += correct
            total_predictions += len(true_labels)

        if cnt == 100:
            break

# 소요 시간 측정 종료
end_time = time.time()

# 소요 시간 및 정확도 출력
time_taken = end_time - start_time

# 정확도 계산
if total_predictions > 0:
    accuracy = (total_correct / total_predictions) * 100
else:
    accuracy = 0

# 소요 시간 및 정확도 출력
print(f"소요 시간: {time_taken:.2f}초")
print(f"정확도: {accuracy:.2f}%")

loading annotations into memory...
Done (t=0.80s)
creating index...
index created!


Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
requirements: Ultralytics requirement ['gitpython>=3.1.30'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 299.2 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 2.7s, installed 1 package: ['gitpython>=3.1.30']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



YOLOv5 🚀 2024-9-24 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

100%|██████████| 14.1M/14.1M [00:00<00:00, 118MB/s] 

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


소요 시간: 28.63초
정확도: 0.28%


### FasterRCNN: 소요 시간: 58.27초, 정확도: 20.46%

### YOLOv5: 소요 시간: 28.63초, 정확도: 0.28%

#### FasterRCNN의 경우 Two-stage 모델이기 때문에 후보 영역을 먼저 선발한 뒤에 이 영역을 세밀하게 분석하므로 정확도가 높지만 소요 시간 역시 많이 걸리게 된다. 그러나 YOLOv5의 경우 One-stage 모델이므로 한 번에 모든 처리를 완료하여 시간이 상대적으로 적게 걸리지만 정확도 역시 낮아지는 모습을 확인할 수 있다.